In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import pyarabic.araby as araby


In [ ]:
# Load the data
# Specify the file path
file_path = "./dataset/train.txt"

# Read the contents of the file
with open(file_path, 'r', encoding='utf-8') as file:
    data_content = file.read()

# Split the data into lines
X_train = data_content.split('\n')

# def remove_non_arabic(text):
#     # Use regex to keep only Arabic letters, spaces, tashkeel, and longation
#     arabic_letters = re.sub(r'[^ ؀-ۿ\u0617-\u061A\u064B-\u0652\u0640]', '', text)
#     # Remove tasheel
#     arabic_letters = re.sub(r'[\u064B-\u0652]', '', arabic_letters)
#     # Remove longation
#     arabic_letters = re.sub(r'[\u0640]', '', arabic_letters)
#     # Remove 3alamat tarkeem (.  ,  ~  :  !  ?  ;  ؟)
#     arabic_letters = re.sub(r'[\.\,\~\:\!\؟\؛]', '', arabic_letters)
#     # Remove extra spaces
#     arabic_letters = re.sub(r'\s+', ' ', arabic_letters)
    
#     return arabic_letters


# # Remove non-Arabic letters
# X_train = [remove_non_arabic(text) for text in X_train]



# print the first 2:6 lines in different lines
# for i in range(2,6):
#     print(X_train[i])


In [ ]:
# def extract_arabic_diacritics(text):
#     """Extracts Arabic diacritics from the given text.

#     Args:
#         text: The Arabic text from which to extract diacritics.

#     Returns:
#         A list of the extracted diacritics.
#     """

#     diacritics_pattern = r"[\u064B-\u065F]"  # Matches all Arabic diacritics
#     # Remove all non-Arabic letters
#     text = re.sub(r'[\u064B-\u065F]', '', text)
    
#     diacritics = re.findall(diacritics_pattern, text)
#     return text,diacritics

# extract_arabic_diacritics(X_train[2])

In [ ]:
def remove_diacritics(text):
    # Remove diacritics
    text = araby.strip_tashkeel(text)
    # Remove longation
    text = araby.strip_tatweel(text)
    # Remove tatweel
    text = araby.strip_shadda(text)
    return text

def remove_non_arabic(text):
    # Remove any non-Arabic letters and leave spaces, leave arabic letters with diacritics
    text=
    text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text



def preprocessing_text(text):
    # Remove diacritics
    text = remove_diacritics(text)
    
    # Remove non-Arabic letters
    text = remove_non_arabic(text)

    # Tokenize
    text = araby.tokenize(text)
    return text

def save_gold_output(text):
    # Remove non-Arabic letters
    text = remove_non_arabic(text)

    # Tokenize
    text = araby.tokenize(text)

    # Write the gold output to a file
    with open('gold_output.txt', 'w', encoding='utf-8') as file:
        for word in text:
            file.write(word + '\n')

test = "قَالَ ابْنُ الْقَاسِمِ : قَالَ مَالِكٌ فِي مَكِّيٍّ أَحْرَمَ بِحَجَّةٍ مِنْ الْحَرَمِ ثُمَّ أُحْصِرَ ، أَنَّهُ يَخْرُجُ إلَى الْحِلِّ فَيُلَبِّي مِنْ هُنَاكَ لِأَنَّهُ أَمَرَ مَنْ فَاتَهُ الْحَجُّ وَقَدْ أَحْرَمَ مِنْ مَكَّةَ ، أَنْ يَخْرُجَ إلَى الْحِلِّ فَيَعْمَلَ فِيمَا بَقِيَ عَلَيْهِ مَا يَعْمَلُ الْمُعْتَمِرُ وَيُحِلُّ .( 2 / 437 ) "
save_gold_output(test)
text = preprocessing_text(test)

In [ ]:
# Important functions in araby

# araby.tokenize(text) # Tokenize the text
# araby.is_arabicrange(text) # Check if the text is Arabic
# araby.sentence_tokenize(text) # Tokenize the text into sentences
# araby.strip_tashkeel(text) # Remove taskeel (ex. fatha, damma, kasra)
# araby.strip_diacritics(text) # remove diacritics (ex. fatha, damma, kasra, shadda)
# araby.strip_tatweel(text) # Remove longation (tatweel)
# araby.strip_shadda(text) # Remove tatweel (shadda)
# araby.auto_correct(text) # Auto correct the text 
# print(araby.arabicrange()) # Arabic letters range
# araby.fix_spaces(text) # Fix spaces

# Deh btfsel ben al tashkeel we al kalam we btrag3hom , btrag3 al text then al tashkeel then al shada law 3amlna extract shada = True
# text , marks ,shada = araby.separate("قَالَ ابْنُ الْقَاسِمِ : قَالَ مَالِكٌ فِي مَكِّيٍّ أَحْرَمَ بِحَجَّةٍ مِنْ الْحَرَمِ ثُمَّ أُحْصِرَ ، أَنَّهُ يَخْرُجُ إلَى الْحِلِّ فَيُلَبِّي مِنْ هُنَاكَ لِأَنَّهُ أَمَرَ مَنْ فَاتَهُ الْحَجُّ وَقَدْ أَحْرَمَ مِنْ مَكَّةَ ، أَنْ يَخْرُجَ إلَى الْحِلِّ فَيَعْمَلَ فِيمَا بَقِيَ عَلَيْهِ مَا يَعْمَلُ الْمُعْتَمِرُ وَيُحِلُّ .( 2 / 437 )",extract_shadda=True) # Separate diacritics from the text
# print (text)
# marks.encode('utf8')
# for m in marks:
#     print (araby.name(m))

# # btgeeb ya shada ya tatwel
# shada.encode('utf8')
# for s in shada:
#     print (araby.name(s))